In [1]:
%load_ext watermark

import pandas as pd
import numpy as np
import datetime as dt 
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import beta
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown as md
from myst_nb import glue

dfCodes = pd.read_csv("resources/data/u_codes.csv")
dfBeaches = pd.read_csv("resources/data/u_beaches.csv")
dfBeaches.set_index("slug", inplace=True)

all_data = pd.read_csv("resources/data/u_all_data.csv")
all_data = all_data[all_data.river_bassin != 'les-alpes'].copy()
all_data["date"] = pd.to_datetime(all_data["date"], format="%Y-%m-%d")

# import regional labels. labels are used
# to identify the regional priors
lac_leman_regions = pd.read_csv("resources/data/lac_leman_regions.csv")

# map to code decriptions
dfCodes.set_index("code", inplace=True)
dfCodes.loc["Gcaps", ["material", "description", "groupname"]] = ["Plastic", "Plastic bottle lids", "food and drink"]

In [2]:
# this defines the css rules for the note-book table displays
header_row = {'selector': 'th:nth-child(1)', 'props': f'background-color: #FFF; text-align:right'}
even_rows = {"selector": 'tr:nth-child(even)', 'props': f'background-color: rgba(139, 69, 19, 0.08);'}
odd_rows = {'selector': 'tr:nth-child(odd)', 'props': 'background: #FFF;'}
table_font = {'selector': 'tr', 'props': 'font-size: 10px;'}
table_data = {'selector': 'td', 'props': 'padding: 12px;'}
table_caption = {'selector': 'caption', 'props': 'font-size: 14px; font-style: italic; caption-side: bottom; text-align: left; margin-top: 10px'}
table_css_styles = [even_rows, odd_rows, table_font, header_row, table_caption]


table_large_data = {'selector': 'tr', 'props': 'font-size: 14px; padding: 12px;'}
table_large_font = [even_rows, odd_rows, table_large_data, header_row, table_caption]

# Testing 2023 predictions

## The solid waste experience

This is the seventh year that the Solid Waste Team from the EPFL collect beach litter samples. In the maritime environment people have been measuring beach litter for decades. There is a standard protocol ([Guidance on Monitoring Marine Litter in European Seas](https://publications.jrc.ec.europa.eu/repository/handle/JRC83985)) for the EU area and threshold values for good environmental standing ([Beach litter thresholds](https://mcc.jrc.ec.europa.eu/main/dev.py?N=41&O=454)). 

In Switzerland we started monitoring shoreline trash in 2015, it was not obvious to most observers (except for Prof Ludwig) why this might be of interest. However, by 2016 the EU realized that monitoring trash flows in rivers and lakes ([monitoring trash in rivers](https://mcc.jrc.ec.europa.eu/documents/201703034325.pdf)) might be a good way to monitor flows into the oceans. All the while conservationists and biologists have raised concerns about the presence of plastics and diminshing biodiversity. The threshold established by the EU is based on the principle of precaution: _the health effects are unknown, it is prudent to reduce contact with plastics when possible_ ([Beach litter thresholds](https://mcc.jrc.ec.europa.eu/main/dev.py?N=41&O=454)).

### Observations and interpretations

A beach litter survey is a detailed observation of the quantity and type of objects that were found at the beach. This observation is further defined by the time and place it occured. The location of the beach litter survey can be described numerically using a topographical map and some common overlay techniques in QGIS.

The information gathered from the map are part of the _conditions_ that describe a survey location in particular. When beach litter surveys are considered in terms of their shared attributes we can use very simple techniques to find correlations between the conditions and the amount of trash found. For example, we can use Spearmans ranked correlation coefficient to quickly identify topographical attributes where specfic objects tend to accumulate. We wrote an article about it:  ([Near or far](https://hammerdirt-analyst.github.io/landuse/titlepage.html)).

### A unique problem and a unique solution

Trash in the environment is a unique problem. In general we know how an object becomes litter: either on purpose or on accident, people create the conditions that increase the chance that an end of lifecycle object will evade the waste recovery system. Resources are employed to change the behavior of people and therefore improve the chance that an end of lifcycle object will be approriately discarded, _(need reference)_. 

There are public services that are dedicated to collecting inappropriately discarded items. Beach litter surveys are the observed result of the difference between the effect of the systems in place to reduce litter and the amount of litter produced. Indifferent of how that litter was produced or the measures in place to prevent it. Therefore this environmental assessment is reliant on individual observations. We can look to orntithologists and botanists for examples on how to interpret this data.

```{admonition} Asessing the environment:

__What and how much are the volunteers likely to find?__
      
_This is the most honest answer that can be derived from the data._
```

There are 336 observations from 66 locations that describe the conditions under which 73,000 items were found on the 145km shore-line of Lake Geneva. Although this is only a small portion of the lake shore, this is still a good amount of samples in a six year period. It would be difficult to find a comparable stretch of coastline anywhere in the world that has that many samples in seven years. We can use that data to form our opinion of what we might find on October 5th.

```{admonition} Asessing the environment:

We can not tell you how much there is. __Only how much you are likely to find.__
      
What the difference is between the two statements is a philosophical discussion. In reality it may be hard to make such a distinction.
```


### Reducing dimensionality: find the most common

There are 228 different categories of objects. We are interested in what we might find and how likely we are to find it. Therefore we limit the search to items that were previously identified in at least 50% of the surveys AND/OR objects that are distinctive (easy to identify). This accounts for 74% of all objects previously recorded.

In [3]:
def prior_distributions(prior_data: pd.DataFrame = None, start: str = None, end: str = None,
                        xrange: np.array = None, uninformed_prior: np.array = None):
    data_args = {
        'start':start,
        'end':end,
        'data': prior_data,
    }
    prior_pcs = period_pieces(*data_args.values())         

    # get n and k for the prior data
    prior_k, prior_notk, prior_k_n_minus_k = period_k_and_n(prior_pcs, xrange)
   
    # make the likelihood parameters
    lhx = list(zip(prior_k, prior_notk))

    # make the prior distribution
    p_ui, prior_bmean = make_expected(lhx, uninformed_prior, xrange)

    # the uninformed beta approximation of the prior data
    prior_beta = [period_beta(x) for x in prior_k_n_minus_k]
    p_beta= [x.mean() for x in prior_beta]

    results=pd.DataFrame({"x":xrange, "p":p_ui})
    results["pn"] = results.p/results.p.sum()
    
    return np.array(p_ui), np.array(p_beta), prior_k_n_minus_k, results, prior_pcs

def posterior_distribution(lh_data: pd.DataFrame = None, start: str = None, end: str = None,
                           informed_prior: np.array = None, un_informed: np.array = None):
                               
    
    data_args = {
        'start': start,
        'end':end,
        'data': lh_data,   
        }

    period_all = period_pieces(*data_args.values())
    
    pall_k, pall_notk, pall_k_n_minus_k = period_k_and_n(period_all, xrange)
    
    lh_and_informed = np.array(pall_k_n_minus_k) + np.array(informed_prior)
    lhx = list(zip(pall_k, pall_notk))        
    
    probi, probi_beta = make_expected(pall_k_n_minus_k, np.array(informed_prior), xrange)
    grid_prox, grid_prox_beta = make_expected(pall_k_n_minus_k, un_informed, xrange)
    
    # beta distribution 
    pall_beta = [period_beta((x[0]+1, x[1]+1)) for x in pall_k_n_minus_k]
    pall_bmean = [x.mean() for x in pall_beta]
    return np.array(probi), np.array(grid_prox), pall_bmean, period_all
                               
def training_testing_compare(lh_pcs, pcs, post_quants, prior_quants):
    
    total_training = len(pcs) + len(lh_pcs)
    prior_weight = len(pcs)/total_training
    lh_weight = len(lh_pcs)/total_training

    number_of_samples = {"before may 2021": len(pcs), "after may 2021": len(lh_pcs)}
    weights = {"before may 2021":prior_weight, "after may 2021": lh_weight}
    observed_median = {"before may 2021":np.median(pcs), "after may 2021": np.median(lh_pcs)}
    observed_average = {"before may 2021":np.mean(pcs), "after may 2021": np.mean(lh_pcs)}
    observed_25 = {"before may 2021": prior_quants[1], "after may 2021":post_quants[1]}
    observed_75 = {"before may 2021": prior_quants[5], "after may 2021":post_quants[5]}
    index = ["weight all samples", "Number of samples", "Median", "Average", "25th percentile", "75th percentile"]
    components = [weights, number_of_samples, observed_median, observed_average, observed_25, observed_75]
    unks_sum_table = pd.DataFrame(components, index=index).style.format(precision=2).set_table_styles(table_large_font)
    styled = unks_sum_table.format(formatter="{:.0f}", subset=pd.IndexSlice[['Number of samples'], :])
    
    return styled

def predicted_summary(lh_pcs, pcs, prior_quants, median_2024):
    

    predicted = ((lh_pcs <= prior_quants[5])&(lh_pcs >= prior_quants[1])).sum()/len(lh_pcs)
    predicted_94 = ((lh_pcs <= prior_quants[-1])&(lh_pcs >= prior_quants[0])).sum()/len(lh_pcs)
    past_present_future = {
        "Median 2021": np.median(pcs), 
        "Median 2022": np.median(lh_pcs), 
        "Expected sampling median 2024":median_2024,
        "% 2022 in 50% IQR  predicted": predicted,
        "% 2022 in 94% IQR  predicted": predicted_94,
    }
        
    
    ppf = pd.DataFrame(past_present_future, index=["pcs/m"]).T

    return ppf


def make_results_df(prior_df, lh_c, source=None, source_norm=None):
    
    prior_df[source] = lh_c
    prior_df[source_norm] = prior_df[source]/prior_df[source].sum()

    return prior_df

def data_profile(all_data):
    date_min = all_data["date"].min()
    date_max = all_data["date"].max()

    if "location" in all_data.columns:
        nlocations = all_data.location.nunique()
    else:
        nlocations = all_data.slug.nunique()
    ncodes = all_data.code.nunique()
    ncities = all_data.city.nunique()
    quantity = all_data.quantity.sum()
    nsamples = all_data.loc_date.nunique()

    a_profile = dict(
        start = date_min,
        end = date_max,
        nlocations = nlocations,
        ncodes = ncodes,
        ncities = ncities,
        quantity = quantity,
        nsamples = nsamples
    )

    return a_profile

def a_fail_rate(x, total_number_of_samples):
    return x["fail"].sum()/total_number_of_samples


def the_most_abundant(x):
    t = x.groupby("code").quantity.sum().copy()
    t.sort_values("quantity", ascending=False, inplace=True)
    return t  

In [4]:
use_groups =  {
    'Personal hygiene':['G95', 'G96'],
    'Personal consumption':['G30', 'Gcaps', 'G27'],
    'Industrial/professional': ['G67', 'G89', 'G112'],
    'Unknown':['Gfrags', 'Gfoam'],
    'Recreation/sports': ['G70', 'G32'],
    
}

use_groups_i =  {
    'G95':'Personal hygiene',
    'G96': 'Personal hygiene',
    'G30':'Personal consumption',
    'Gcaps':'Personal consumption',
    'G27':'Personal consumption',
    'G67':'Industrial/professional',
    'G89':'Industrial/professional',
    'G112': 'Industrial/professional',
    'Gfoam':'Unknown',
    'Gfrags':'Unknown',
    'G70':'Recreation/sports',
    'G32':'Recreation/sports',
}

abbrev_use_g = {'Unknown':'Unk','Personal consumption':'Pc', 'Personal hygiene': 'Ph',    'Recreation/sports': 'Rc', 'Industrial/professional':'Ip'}
toi = list(use_groups_i.keys())


In [5]:
cbdi = pd.read_csv("resources/data/u_pstk_iqaasl_all.csv")
not_these = ['amphion', 'anthy', 'excenevex', 'lugrin', 'meillerie', 'saint-disdille', 'tougues']
cbd = cbdi[~cbdi.slug.isin(not_these)]
cbd = cbd[cbd.code.isin(toi)].copy()
cbd["fail"] = cbd.quantity > 0


cbd.loc[cbd.Project == "Testing", "Project"] = "after may 2021"
cbd.loc[cbd.Project == "Training", "Project"] = "before may 2021"

column_names_groups = {v:k for k,v in abbrev_use_g.items()}
code_groups = list(column_names_groups.keys())

cois = cities_of_interest = ['Saint-Sulpice (VD)', 'Saint Gingolph', 'Genéve', 'Cully', 'Vevey']

some_quants = [.03, .25, .48, .5, .52, .75, .97]
end_training_date = "2021-05-31"
begin_training_date = "2015-11-15"
codes_of_interest = cbd.groupby(["code"], as_index=False).agg({"quantity":"sum", "pcs/m":"mean", "fail": "sum"})
codes_of_interest["fail rate"] = (codes_of_interest.fail/cbd.loc_date.nunique()).round(2)
code_d = dfCodes["description"]
codes_of_interest["object"] = codes_of_interest.code.apply(lambda x: code_d.loc[x])
codes_of_interest = codes_of_interest[["code", "object", "pcs/m", "quantity", "fail rate"]]
codes_of_interest.set_index(["code", "object"], inplace=True, drop=True)
codes_of_interest["quantity"] = codes_of_interest.quantity.astype("int")
codes_of_interest["% of total"] = (codes_of_interest.quantity/cbdi.quantity.sum()).round(2)
codes_of_interest.index.name = None
caption = "Table 1: The objects of interest. The average pcs/m per sample for each object. The fail rate is the % of all samples that the object appeared in." 
codes_of_interest.style.format(precision=2).set_table_styles(table_large_font).set_caption(caption)

,,pcs/m,quantity,fail rate,% of total
code,object,,,,
G112,Industrial pellets (nurdles),0.16,2686,0.22,0.02
G27,Cigarette filters,1.12,16458,0.85,0.15
G30,"Food wrappers; candy, snacks",0.54,6767,0.86,0.06
G32,Toys and party favors,0.05,606,0.48,0.01
G67,Industrial sheeting,0.30,3356,0.57,0.03
G70,Shotgun cartridges,0.08,1030,0.48,0.01
G89,Plastic construction waste,0.14,1970,0.51,0.02
G95,Cotton bud/swab sticks,0.39,4777,0.74,0.04
G96,Sanitary pads /panty liners/tampons and applicators,0.04,373,0.29,0.00


### Assessing the environment

{Download}`Download the form </resources/figures/survey_estimates.pdf>`

The goal for todays excercise in 2023 is to determine how well our previous experiences inform us about the present. This is a simple process. There are four steps:

1. Start with your current understanding of the problem, consult the data here and form an opinion of how many of each item in the previous section you might find in 100 meters of shoreline. For example your might think 200 cigarette ends per 100m is a likely amount.
2. Use the provided form and note your estimate for each item in red ink. Put your name on the form and the name of the beach.
3. At the end of the litter survey note what you found for each item.

After the survey we will compare what we found to what we though we might find and the predicted amount using the model that was explained in the previous section. 

### Semester project

The semester project (if you choose to do it) is about documenting the process of updating the models and accessing data. It could be a narrated screencast. Something that next years class will consult. For those who are interested in data-science or application development we would be using python, R, Git and Annaconda. If you do not know what those things are then the project is not for you. 

However, if you have done a data-science course or if you have some experience with application development you might find this an easy project that will allow you to demonstrate those skills and some creativity. Those that know how to use Git and Annaconda will find this fairly easy.

(data-context)=
## Summary of previous results

__Lake Geneva sample totals__

The total pcs/m for all surveys is given in figure 1 and figure 2. Samples after May 2021 are considered separately, this is a new six year sampling period for the lake.  The distribution of the sample totals is given in table 2.

In [6]:
summ_data = cbd.copy()

summ_data["use group"] = summ_data.code.map(lambda x: use_groups_i[x])

summ_data["ug"] = summ_data["use group"].apply(lambda x: abbrev_use_g[x])
summ_data[summ_data["use group"] == 'Personal consumption'].code.unique()
summ_data["date"] = pd.to_datetime(summ_data["date"], format="%Y-%m-%d")

sd_x = summ_data.groupby(["loc_date", "date", "city", "Project", "doy"], as_index=False).agg({"pcs/m": 'sum', 'quantity':'sum'})
sd_x_sp = sd_x[sd_x.city == 'Saint-Sulpice (VD)'].groupby(["loc_date", "date", "city", "Project", "doy"], as_index=False).agg({"pcs/m": 'sum', 'quantity':'sum'})

trg = summ_data[summ_data.Project == "before may 2021"].copy()
tst = summ_data[summ_data.Project == "after may 2021"].copy()
trg_c, tst_c = trg.city.nunique(), tst.city.nunique()
trg_lc, tst_lc = trg.slug.nunique(), tst.slug.nunique()
trg_q, tst_q = trg.quantity.sum(), tst.quantity.sum()

data_magnitude = [
    {"before may 2021":trg_c, "after may 2021":tst_c},
    {"before may 2021":trg_lc, "after may 2021":tst_lc},
    {"before may 2021":trg_q, "after may 2021":tst_q}
    
]

cities_set = list(set([*trg.city.unique(), *tst.city.unique()]))
n_ind_cities = len(cities_set)

caption = f'The number of different locations and cities for the data. Note that there are {n_ind_cities} different municipalitites in all.'

data_summ_q = pd.DataFrame(data_magnitude, index=["Number of cities", "Number of locations", "Total objects"]).astype('int')
data_summ_q = data_summ_q.style.format(formatter="{:,}").set_table_styles(table_large_font).set_caption(caption)
styled = data_summ_q.format(formatter="{:,}", subset=pd.IndexSlice[['Total objects'], :])
glue("data-summ-q2", styled, display=False)

In [7]:
# all the data by date
the_99th_percentile = np.quantile(sd_x['pcs/m'].values, .99)
px = 1/plt.rcParams['figure.dpi']  # pixel in inches
fig, ax = plt.subplots(figsize=(600*px,500*px))

sns.scatterplot(data=sd_x, x='date', y='pcs/m',ax=ax, color="dodgerblue", alpha=0.6,label="lac léman")
sns.scatterplot(data=sd_x_sp, x='date', y='pcs/m', color="magenta", label="solid-waste-team", ax=ax)

ax.set_ylim(-1, the_99th_percentile)
ax.legend(loc="upper left")
ax.set_xlabel("")
glue("testing_training_chrono", fig, display=False)
plt.close()

In [8]:
# all the data day of year
fig, ax = plt.subplots(figsize=(600*px, 500*px))

sns.scatterplot(data=sd_x, x='doy', y='pcs/m', ax=ax, color="dodgerblue", alpha=0.6,label="lac léman")
sns.scatterplot(data=sd_x_sp, x='doy', y='pcs/m', color="magenta", label="solid-waste-team", ax=ax)
ax.set_ylim(-1, the_99th_percentile)
ax.set_xlabel("Day of the year")
glue('testing_training_doy', fig, display=False)
plt.close()

In [9]:
testing_vals= sd_x[sd_x.Project == "after may 2021"]['pcs/m'].values
training_vals = sd_x[sd_x.Project == "before may 2021"]['pcs/m'].values


train_quantiles = np.quantile(training_vals, some_quants)
test_quantiles = np.quantile(testing_vals, some_quants)

training_testing_summary = training_testing_compare(testing_vals, training_vals, test_quantiles, train_quantiles)
caption = "The observed values from the training and testing data. Remark that the testing data is only 22% of all the data. This is because we are only in the first year of a six year sampling period"
sum_table = training_testing_summary.set_caption(caption)
sum_table.format(formatter="{:.0f}", subset=pd.IndexSlice[['Number of samples'], :])
glue("data-summary", sum_table, display=False)

In [10]:
fig, ax = plt.subplots(figsize=(600*px, 500*px))

sns.ecdfplot(data=sd_x, x='pcs/m', hue='Project',  hue_order=["before may 2021", "after may 2021"],ax=ax)
ax.set_xlim(-1, the_99th_percentile)
ax.set_ylabel("Cumulative probability")
glue('testing_training_cumulative', fig, display=False)
plt.close()

|Figure 1, Table 1 | Table 2, Figure 2|
|:-----------------------:|:---------------------:|
|{glue:}`testing_training_chrono` |{glue}`data-summary`|
|{glue:}`data-summ-q2`|{glue}`testing_training_doy`|

In [11]:
summ_data = cbd.copy()

summ_data["use group"] = summ_data.code.map(lambda x: use_groups_i[x])

summ_data["ug"] = summ_data["use group"].apply(lambda x: abbrev_use_g[x])
summ_data[summ_data["use group"] == 'Personal consumption'].code.unique()
summ_data["date"] = pd.to_datetime(summ_data["date"], format="%Y-%m-%d")

sd_x = summ_data.groupby(["loc_date", "date", "city", "Project", "doy"], as_index=False).agg({"pcs/m": 'sum', 'quantity':'sum'})
sd_x_sp = sd_x[sd_x.city == 'Saint-Sulpice (VD)'].groupby(["loc_date", "date", "city", "Project", "doy"], as_index=False).agg({"pcs/m": 'sum', 'quantity':'sum'})

trg = summ_data[summ_data.Project == "before may 2021"].copy()
tst = summ_data[summ_data.Project == "after may 2021"].copy()
trg_c, tst_c = trg.city.nunique(), tst.city.nunique()
trg_lc, tst_lc = trg.slug.nunique(), tst.slug.nunique()
trg_q, tst_q = trg.quantity.sum(), tst.quantity.sum()

data_magnitude = [
    {"before may 2021":trg_c, "after may 2021":tst_c},
    {"before may 2021":trg_lc, "after may 2021":tst_lc},
    {"before may 2021":trg_q, "after may 2021":tst_q}
    
]

cities_set = list(set([*trg.city.unique(), *tst.city.unique()]))
n_ind_cities = len(cities_set)

caption = f'The number of different locations and cities for the data. Note that there are {n_ind_cities} different municipalitites in all.'

data_summ_q = pd.DataFrame(data_magnitude, index=["Number of cities", "Number of locations", "Total objects"]).astype('int')
data_summ_q = data_summ_q.style.format(formatter="{:,}").set_table_styles(table_large_font).set_caption(caption)
styled = data_summ_q.format(formatter="{:,}", subset=pd.IndexSlice[['Total objects'], :])
glue("data-summ-q2", styled, display=False)

In [12]:
def sampler_from_multinomial(normed, xrange, nsamples):
    
    choose = np.random.default_rng()
    nunique = np.unique(normed)
    norm_nunique = nunique/np.sum(nunique)
    found = choose.multinomial(1, pvals=norm_nunique, size=nsamples)
    ft = found.sum(axis=0)
    samples = []
    for i, asum in enumerate(ft):
        if asum == 0:
            samples += [0]
        else:
            choices = np.where(normed == nunique[i])
            samps = choose.choice(choices[0], size=asum)
            samples.extend(xrange[samps])

    return samples, nunique, norm_nunique, ft

def period_pieces(start, end, data):
    # the results in pieces per meter for one code from a subset of data
    date_mask = (data["date"] >= start) & (data["date"] <= end)
    period_one = data[date_mask]
    pone_pcs = period_one.pcs_m.values

    return pone_pcs

def period_k_and_n(data, xrange, add_one=False):

    pone_k = [(data >= x).sum() for x in xrange]
    pone_notk = [(data < x).sum() for x in xrange]

    if add_one:
        # if the use is for beta dist. This is the same
        # as mulitplying the likelihood * uninform prior (0.5) or beta(1,1)
        pone_k_n_minus_k = [(x+1, len(data) - x+1) for x in pone_k]
    else:
        pone_k_n_minus_k = [(x, len(data) - x) for x in pone_k]
        
    

    return np.array(pone_k), np.array(pone_notk), np.array(pone_k_n_minus_k)

def period_beta(k):
    
         
    return beta(*k)
        

def current_possible_prior_locations(landuse, locations, attribute):    

    # indentify the magnitude(s) of the attribute of interest from the
    # locations in the current data there may be more than one, in this 
    # example we use all the possible magnitudes for the attribute
    # locations = data[data.city == city].location.unique()

    # magnitudes for the attribute from all the locations in the municipality
    moa = magnitude_of_attribute = landuse.loc[locations][attribute].unique().astype('int')

    # identify locations that have the same attribute by magnitude of attribute
    possible_locations = landuse[landuse[attribute].isin(moa)].index

    # remove the locations that are in the likelihood function
    prior_locations = [x for x in possible_locations if x not in locations]

    return locations, possible_locations, prior_locations


def make_expected(lh_tuple, prior_tuple, xrange):
    res = []
    betas=[]
    # print(lh_tuple, prior_tuple)
    for i in np.arange(len(xrange)):
        alpha = prior_tuple[i][0]
        betai = prior_tuple[i][1]
        success = lh_tuple[i][0]
        n = lh_tuple[i][1] + lh_tuple[i][0] 
        numerator = alpha + success
        denominator = alpha + betai + n
        if numerator == 0:
            numerator = 1
        abeta = beta(numerator, (betai + lh_tuple[i][1] + lh_tuple[i][0])).mean()
        betas.append(abeta)
        # print(alpha, betai, success, numerator, n, denominator)
        if numerator >= denominator:
            numerator = denominator-1
            
        expected = numerator/denominator
        res.append(expected)
    return np.array(res), np.array(betas) 

In [13]:
comb_lu_agg = pd.read_csv("resources/data/u_comb_lu_cover_street_rivers.csv")

lu_scaled = comb_lu_agg.pivot(columns="use", values="scaled", index="slug").fillna(0)

lu_magnitude = comb_lu_agg.pivot(columns="use", values="magnitude", index="slug").fillna(0)

lu_binned = comb_lu_agg.pivot(columns="use", values="binned", index="slug").fillna(0)

# not_these = ['amphion', 'anthy', 'excenevex', 'lugrin', 'meillerie', 'saint-disdille', 'tougues']
merge_locations = cbd.slug.unique()
cbdu = cbd[~cbd.slug.isin(not_these)].merge(lu_scaled[lu_scaled.index.isin(merge_locations )], left_on="slug", right_index=True, validate="many_to_one", how="outer")

cbdu["use group"] = cbdu.code.map(lambda x: use_groups_i[x])

cbdu["ug"] = cbdu["use group"].apply(lambda x: abbrev_use_g[x])
cbdu[cbdu["use group"] == 'Personal consumption'].code.unique()
cbdu["date"] = pd.to_datetime(cbdu["date"], format="%Y-%m-%d")

attribute_columns = [x for x in lu_scaled.columns if x not in ["Geroell", "Stausee", "See", "Sumpf", "Stadtzentr", "Fels"]]
work_columns = [x for x in cbdu.columns if x not in ["Geroell", "Stausee", "See", "Sumpf", "Stadtzentr", "Fels"]]
cbdu = cbdu[work_columns].copy()
cbdu.rename(columns={"pcs/m":"pcs_m"}, inplace=True)


## Expected survey results Saint Sulpice

The expected survey results will be revealed once the class has made the recomendations.

In [14]:
city =  'Saint-Sulpice (VD)'
start, end = "2015-11-15", "2021-05-31"
g_resa = cbdu.copy()
g_resa = g_resa.groupby(['loc_date', 'date','slug', 'city', 'Project', 'ug'], as_index=False).agg({'pcs_m':'sum', 'quantity':'sum'})
g_resa.rename(columns={"ug":"code"}, inplace=True)
g_resadt = g_resa.groupby(['loc_date', 'date','slug', 'city', 'Project'], as_index=False).agg({'pcs_m':'sum', 'quantity':'sum'})

In [15]:
index_range = (0.0, 10)

xrange =  np.arange(*index_range, step=.01)
uninformed_tuple = np.array([(1,1) for x in xrange])
this_attribute = attribute_columns[2]
# define the prior, likelihood data and likelihood locations
# removing the results from saint sulpice from the prior data
prior_data = g_resadt[(g_resadt.Project =="before may 2021")&(g_resadt.city != city)]

# the likelihood data
lh_data = g_resadt[(g_resadt.city == city)].copy()
lh_locations = lh_data.slug.unique()

# find other locations in the regions
regions = lac_leman_regions[~lac_leman_regions.slug.isin(not_these)].copy()
lh_regions = regions[regions.slug.isin(lh_locations)].alabel.unique()
regional_locations = regions[regions.alabel.isin(lh_regions)].slug.unique()

# compare the landuse values of locations in the region
# select locations that are similar to the locations in Saint Sulpice
# use the this_attribute to compare 
land_use_data_of_interest = lu_binned.loc[regional_locations]
locations, possible_locations, prior_locations = current_possible_prior_locations(land_use_data_of_interest, lh_locations, this_attribute)

prior_args = {
    'prior_data':prior_data[prior_data.slug.isin(prior_locations)],
    'start': start,
    'end': end,
    'xrange':xrange,
    'uninformed_prior': uninformed_tuple,
}
# grid approximation of the prior
# this returns a calculated grid approximation
# the beta approximation, the beta parameters
# and the data used to make the inference
grid_prior, beta_prior, prior_k_n, prior_df, pcs = prior_distributions(**prior_args)

posterior_args = {
    'lh_data':lh_data,
    'start': start,
    'end': "2022-12-31",
    'un_informed': uninformed_tuple,
    'informed_prior': prior_k_n
}

# grid approximation of posterior
informed, uninformed, beta_p, lh_pcs = posterior_distribution(**posterior_args)

prior_df["Informed"] = informed
prior_df["Ip_n"] = prior_df.Informed/prior_df.Informed.sum()
prior_df["Uninformed post"] = uninformed

# the quantiles from the observed data
prior_quants = np.quantile(pcs, some_quants)

In [16]:
# the quantiles from the posterior
post_quants = np.quantile(lh_pcs, some_quants)

# samples from posterior
# the grid approximation uses the mean of the binomial
# distribution that describes wether a sample will exceed a given value
# therefore we can use a multinomial distribution for that
choose = np.random.default_rng()
sim_2024, nunique, norm_nunique, ft = sampler_from_multinomial(prior_df["Ip_n"].values, xrange, 200)
x_median_2024 = prior_df.loc[prior_df["Informed"].between(.45, .55), "x"].mean()

# observed in relation to predicted
ppf = predicted_summary(lh_pcs, pcs, prior_quants, x_median_2024)
caption="Previous and expected median values of the combined daily totals of the Unkown objects group"

ppf_d = ppf.style.format(precision=2).set_caption(caption).set_table_styles(table_large_font)
glue("ssp-2024-meds", ppf_d, display=False)

# comparing training to testing
unks_sum_table = training_testing_compare(lh_pcs, pcs, post_quants, prior_quants)
caption = "The observed values from the training and testing data."
sum_table = unks_sum_table.set_caption(caption)
glue("ssp-summary", sum_table, display=False)

In [17]:
fig, ax = plt.subplots()

# points
median_prior = np.median(pcs)
median_lh = np.median(lh_pcs)
quants_2024 = np.quantile(sim_2024, some_quants)

# posterior
ax.plot(xrange, informed, c="magenta", linewidth=3,linestyle=':', zorder=20, label='Outlook 2024')
ax.plot([x_median_2024], [.5],  c="black", markersize=6, marker="x", zorder=27, label="Expected median 2024")
ax.plot(xrange, uninformed, c="darkslategrey",  linestyle=':', linewidth=3, zorder=11, alpha=0.5,  label='2022')
ax.plot([median_lh], [.5], c="blue", markersize=5, marker="o", zorder=25, label="2022 median")
ax.plot(xrange, grid_prior, c="cornflowerblue", linestyle=':',  linewidth=4, zorder=11, alpha=0.5, label="2021")
ax.plot([median_prior], [.5], c="red", markersize=5, marker="o", zorder=25, label="2021 median")

# 50% IQR
ax.axvspan(xmin=prior_quants[1], xmax=prior_quants[5], ymin=0.25, ymax=0.75, facecolor='cornflowerblue', zorder=13, alpha=0.2, label="IQR - 2021")
ax.axvspan(xmin=post_quants[1], xmax=post_quants[5], ymin=0.25, ymax=0.75, facecolor='darkslategrey', zorder=13, alpha=0.2, label="IQR - 2022")
ax.axvspan(xmin=quants_2024[1], xmax=quants_2024[5], ymin=0.25, ymax=0.75, facecolor='black', zorder=15, alpha=0.2, label="IQR - 2024")

ax.set_xlabel('pcs/m')
ax.set_xlim(-.1, 10)
ax.set_ylim(-.01, 1)
ax.set_ylabel('probability')
h, l = ax.get_legend_handles_labels()
ax.legend(h, l, bbox_to_anchor=(0,1.05), loc="lower left", ncol=2 )
glue('ssp-outlook-2024', fig, display=False)
plt.close()

In [18]:
fig, ax = plt.subplots()

unin_2022 = prior_df["Uninformed post"].values

# hists = pd.DataFrame({"Observed 2021": pcs, "Expected 2024": sim_2024, "Observed 2022": lh_pcs}, index=xrange)

sns.ecdfplot([*lh_pcs, *pcs], label="Observed 2015 - 2022",  c="black", stat="proportion", ax=ax, zorder=10)
sns.ecdfplot(pcs, ax=ax, label="Observed 2015 - 2021",color="cornflowerblue", stat="proportion", zorder=10)
sns.ecdfplot(lh_pcs, ax=ax, label="Observed 2022", color="darkslategrey", stat="proportion", zorder=10)
# sns.ecdfplot(sim_2024, ax=ax, label="Observed 2022", color="magenta", stat="proportion", zorder=11)

# ax.plot(xrange, grid_prior, linestyle=':', c="blue", linewidth=4,label="Grid prior", zorder=11)
ax.plot(xrange, (1-informed), linestyle=':', c="magenta", linewidth=3,label="Expected 2024", zorder=11)
# ax.plot(xrange, unin_2022, linestyle=':', c="purple", linewidth=4, label="Grid uninformed 2022", zorder=11)
# ax.hlines(y=0.5, xmin=0, xmax=2, color="r", linestyle="-.", zorder=15)

# sns.histplot(lh_pcs, ax=ax, label="observed 2022", stat="probability")
sns.histplot(pcs, ax=ax, label="Observed 2021", color="cornflowerblue", alpha=0.5, zorder=0, stat="probability")
sns.histplot(sim_2024, ax=ax, label="Expected 2024", color="black", zorder=2, alpha=0.5, edgecolor="magenta", linewidth=1, stat="probability")
sns.histplot(lh_pcs, ax=ax, label="Observed 2022", color="darkslategrey", alpha=0.5, stat="probability", zorder=0)


ax.set_xlabel('pcs/m')
ax.set_xlim(-.1, 10)
ax.set_ylabel('probability')
# h, l = ax.get_legend_handles_labels()
ax.legend(bbox_to_anchor=(0,1.05), loc="lower left", ncol=2 )
glue('ssp-predicted_samples', fig, display=False)
plt.close()

<!-- ### Previous and expected survey totals -->

<!-- |     Figure 11, Table 11  |    Table 12, Figure 12       | 
|:------------------------:|:----------------------------:|
|{glue:}`ssp-outlook-2024` | {glue:}`ssp-2024-meds`|
|{glue:}`ssp-summary` | {glue:}`ssp-predicted_samples`| -->



In [19]:
city =  'Saint-Sulpice (VD)'
start, end = "2015-11-15", "2021-05-31"

g_resa = cbdu.copy()
g_resa = g_resa.groupby(['loc_date', 'date','slug', 'city', 'Project', 'code'], as_index=False).agg({'pcs_m':'sum', 'quantity':'sum'})
g_resadt = g_resa.groupby(['loc_date', 'date','slug', 'city', 'Project'], as_index=False).agg({'pcs_m':'sum', 'quantity':'sum'})

# define the prior, likelihood data and likelihood locations
posterior_df = pd.DataFrame(index=xrange)
predictions = {}

for code in toi:
    
    # code_index = 1
    city_index = 0
    attribute_index = 2
    
    this_code =  code
    this_attribute = attribute_columns[attribute_index]
    this_city = cois[city_index]
    
    prior_data = g_resa[(g_resa.code == this_code)&(g_resa.city != city)&(g_resa.Project =="before may 2021")]
    lh_data = g_resa[(g_resa.code == this_code)&(g_resa.city == city)]
    lh_locations = lh_data.slug.unique()
    
    regions = lac_leman_regions[~lac_leman_regions.slug.isin(not_these)].copy()
    lh_regions = regions[regions.slug.isin(lh_locations)].alabel.unique()
    regional_locations = regions[regions.alabel.isin(lh_regions)].slug.unique()
    land_use_data_of_interest = lu_binned.loc[regional_locations]
    
    locations, possible_locations, prior_locations = current_possible_prior_locations(land_use_data_of_interest, lh_locations, this_attribute)
    
    prior_args = {
        'prior_data':prior_data[prior_data.slug.isin(prior_locations)],
        'start': start,
        'end': end,
        'xrange':xrange,
        'uninformed_prior': uninformed_tuple,
    }
    # grid approximation of the prior
    grid_prior, beta_prior, prior_k_n, prior_df, pcs = prior_distributions(**prior_args)
    
    posterior_args = {
        'lh_data':lh_data,
        'start': start,
        'end': "2022-12-31",
        'un_informed': uninformed_tuple,
        'informed_prior': prior_k_n
    }
    
    # grid approximation of posterior
    informed, uninformed, beta_p, lh_pcs = posterior_distribution(**posterior_args)
    
    # the quantiles from the observed data
    prior_quants = np.quantile(pcs, some_quants)
    post_quants = np.quantile(lh_pcs, some_quants)
    
    # data frame with normalized results
    post_df = make_results_df(prior_df.copy(), informed, source="Informed post", source_norm="Ip_n")
    post_df = make_results_df(post_df, uninformed, source="Uninformed post", source_norm="Un_n")
    
    # samples from posterior 
    sim_2024 = sampler_from_multinomial(post_df["Ip_n"].values, xrange, len(pcs) + len(lh_pcs))
    sim_quants = np.quantile(sim_2024[0], some_quants)
    
    
    predictions.update({this_code:sim_quants})
    posterior_df[this_code]=informed

index = ['{:.0%}'.format(x) for x in some_quants]
pred_quants = pd.DataFrame(predictions, index=index)

In [20]:
pred_quants.style.format(precision=2).set_table_styles(table_large_font)

,G95,G96,G30,Gcaps,G27,G67,G89,G112,Gfoam,Gfrags,G70,G32
3%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.02,0.01,0.00,0.02,0.02,0.01,0.00,0.00,0.00,0.00,0.00
48%,0.32,0.04,0.13,0.08,0.43,0.13,0.08,0.01,0.13,0.51,0.00,0.03
50%,0.33,0.04,0.13,0.12,0.46,0.15,0.10,0.01,0.20,0.69,0.00,0.04
52%,0.33,0.04,0.13,0.12,0.48,0.16,0.11,0.01,0.21,0.77,0.00,0.04
75%,0.59,0.10,0.49,0.21,0.92,0.30,0.20,0.23,0.70,0.96,0.04,0.07
97%,1.11,0.20,0.99,0.70,1.97,0.86,0.39,1.03,2.53,2.01,0.07,0.14


## Discussion

## Conclusions

To be completed on the 6th of October.

### Next steps

Make hierarchical model

In [21]:
today = dt.datetime.now().date().strftime("%d/%m/%Y")
where = "Biel, CH"

my_block = f"""

This script updated {today} in {where}

\u2764\ufe0f __what you do everyday:__ *analyst at hammerdirt*
"""

md(my_block)



This script updated 05/10/2023 in Biel, CH

❤️ __what you do everyday:__ *analyst at hammerdirt*


In [22]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/patelmanuscript.git

Git branch: newsummary

matplotlib: 3.7.1
seaborn   : 0.12.2
numpy     : 1.24.3
pandas    : 2.0.2

